### <strong>Preprocesarea setului de date pentru Algoritmul 3</strong>

Vrem să păstrăm din setul de date doar imaginea, sentimentul corespunzător și gradul de încredere al sentimentului ales.

#### Funcția pentru thread în care testăm dacă imaginea de la url-ul respectiv este una validă

In [ ]:
import PIL
import requests
from PIL import Image
from io import BytesIO


def parse_file(url, data, idx, mutex):
    try:
        response = requests.get(url)
        Image.open(BytesIO(response.content))
    except PIL.UnidentifiedImageError:
        mutex.acquire()
        print(url)
        data.drop(idx, inplace=True)
        data.to_csv("data/image-sentiment-polarity.csv", index=False)
        mutex.release()
        return None

#### Funcția care creează thread-ul care va procesa informația

In [ ]:
from threading import Thread


def read_image_from_url(url: str, data, idx, mutex):
    thread = Thread(target=parse_file, args=(url, data, idx, mutex))
    thread.start()
    return thread

#### Se pornesc batch-uri de 2000 de thread-uri care verifică dacă imaginile sunt valide

In [ ]:
import pandas as pd
from threading import Lock


data = pd.read_csv("data/image-Sentiment-polarity-DFE.csv")
data = data[['which_of_these_sentiment_scores_does_the_above_image_fit_into_best', 'which_of_these_sentiment_scores_does_the_above_image_fit_into_best:confidence', 'imageurl']]
data = data.dropna()
data = data.drop_duplicates()
data.to_csv("data/image-sentiment-polarity.csv", index=False)
threads = []
mutex = Lock()
for idx, row in data.iterrows():
    threads.append(read_image_from_url(row['imageurl'], data, idx, mutex))
    if idx % 2000 == 0 and idx > 0:
        for thread in threads:
            thread.join()
        threads = []